# Time series prediction

## 1) Importing Necessary libraries

In [18]:
import pickle
from datetime import date , datetime
import mysql.connector


## 2) Importing the model as a pickle file

In [19]:
filename = 'modelForIOT.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

## 3) Creating a connection to the database

In [20]:
conn = mysql.connector.connect(
    host="localhost",
    database="iotdba_2022_05_database",
    user="root",
    password="" ,
    port = "3308")

cursor = conn.cursor()

## 4) Getting existing data for validation from the database

In [21]:
cursor.execute("select * from predicts")
f = []
g = []
for i in cursor:
    f.append(str(i[0]))
    g.append(i[1])

## 5) Arranging the current date in required format for prediction 

In [22]:
today = date.today()
# dd/mm/YY
d1 = today.strftime("%m/%d/%Y")
s = d1.split("/")
s[2] = int(s[2])+1
new = s[0]+"/"+str(1)+"/"+str(s[2])

## 6) Prediction

In [23]:
predictions = loaded_model.predict('10/01/2013',new)
predictions

d:\pyenv\penv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


2013-10-01    28.236614
2013-11-01    28.560147
2013-12-01    27.446360
2014-01-01    26.023771
2014-02-01    25.723927
                ...    
2022-11-01    28.556746
2022-12-01    27.467595
2023-01-01    26.074717
2023-02-01    25.770663
2023-03-01    26.858811
Freq: MS, Length: 114, dtype: float64

## Inserting into the database

#### If the prediction for a specific date exists it will be checked if the temperature in the database and the newly predicted value is the same
#### If same no change will be done
#### If changed the value will be updated in the database

In [24]:
for i,j in predictions.items():
    my_date = datetime.strftime(i, '%Y-%m-%d')
    if my_date not in f:
        cursor.execute("INSERT INTO `predicts` (`date`, `temperature`) VALUES ('"+ my_date +"', "+ str(j) +")")
    elif round(j,4) not in g:
        cursor.execute("UPDATE `predicts` SET `temperature` = '"+str(j)+"' WHERE `predicts`.`date` = '"+ my_date +"'")